In [1]:
from langgraph.graph import StateGraph,START,END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.runnables import RunnableConfig
from operator import add


In [2]:
class State(TypedDict):
    foo:str
    bar:Annotated[list[str],add]
    

In [3]:
def node_a(state:State):
    return {
        "foo":"a",
        "bar":["a"]
    }

In [4]:
def node_b(state:State):
    return {
        "foo":"b",
        "bar":["b"]
    }

In [5]:
workflow=StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)

workflow.add_edge(START,"node_a")
workflow.add_edge("node_a","node_b")
workflow.add_edge("node_b",END)

checkpointer=InMemorySaver()
graph=workflow.compile(checkpointer=checkpointer)
config:RunnableConfig={"configurable":{"thread_id":"1"}}


In [ ]:
graph.invoke({
    "foo":"",
    "bar":[]
},config)

{'foo': 'b', 'bar': ['a', 'b', 'a', 'b']}

In [7]:
#get the latest state of snapshot
config={"configurable":{"thread_id":"1"}} # in case you don't remember name do this
graph.get_state(config)

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd101-2fe7-6db9-8002-263b3688fcf9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-29T12:43:12.845755+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd101-2fe6-655c-8001-14b6636419b0'}}, tasks=(), interrupts=())

In [8]:
# get a state snapshot for a specific checkpoint_id
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f0fd0e7-04c1-6a61-8002-5969a92313c1"}}
graph.get_state(config)

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0fd0e7-04c1-6a61-8002-5969a92313c1'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

GET STATE HISTORY

In [9]:
config={"configurable":{"thread_id":"1"}}
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd101-2fe7-6db9-8002-263b3688fcf9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-29T12:43:12.845755+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd101-2fe6-655c-8001-14b6636419b0'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd101-2fe6-655c-8001-14b6636419b0'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2026-01-29T12:43:12.845128+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd101-2fe3-6800-8000-342cf6b8cc89'}}, tasks=(PregelTask(id='8e0b70c7-2de9-51c4-597c-e5fc386b58a5', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts

REPLAY

In [15]:
config={"configurable":{"thread_id":"1","checkpoint_id":"1f0fc700-5231-6256-8000-bbe76850b6b6"}}
graph.invoke(None,config=config)

{'foo': 'b', 'bar': ['a', 'b']}

In [ ]:
# Replay behavior explanation:
# When invoking a graph with a thread_id and checkpoint_id, LangGraph checks
# which steps (nodes) were already executed in that thread.
#
# - For steps BEFORE the given checkpoint_id:
#   LangGraph does NOT re-run them.
#   Instead, it replays their previously stored outputs from the checkpoint history.
#
# - For steps AFTER the checkpoint_id:
#   LangGraph ALWAYS executes them again, even if they were executed before.
#   This creates a new execution path (a fork) starting from the checkpoint.
#
# In short:
#   Past steps → replayed (cached results)
#   Future steps → re-executed (new branch)
